# Loading and Pre-processing Data

In [1]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.download()
nltk.data.path.append('.')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [2]:
with open("en_US.twitter.txt", "r", encoding = "utf8") as f:
    data = f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("\nFirst 300 letters of the data")
print("-------")
display(data[0:300])
print("-------")
print("\nLast 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")

Data type: <class 'str'>
Number of letters: 3335476

First 300 letters of the data
-------


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

-------

Last 300 letters of the data
-------


"just had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea."

-------


In [3]:
def split_to_sentences(data):
    sentences = data.split('\n')
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]
    
    return sentences   

In [4]:
x = """
My name is Abhinav.\nI am an Undergraduate Student.\nI study at XXX College of Engineering.\nI am in 3rd Year.
"""
print(x)
split_to_sentences(x)


My name is Abhinav.
I am an Undergraduate Student.
I study at XXX College of Engineering.
I am in 3rd Year.



['My name is Abhinav.',
 'I am an Undergraduate Student.',
 'I study at XXX College of Engineering.',
 'I am in 3rd Year.']

In [5]:
def tokenize_sentences(sentences):
    tokenized_sentences = []
    
    for sentence in sentences:
        sentence = sentence.lower()
        tokenized = nltk.word_tokenize(sentence)
        tokenized_sentences.append(tokenized)
    
    return tokenized_sentences

In [6]:
sentences = ["I am Abhinav.", "I am an Undergraduate Student.", "I am in 3rd Year."]
tokenize_sentences(sentences)

[['i', 'am', 'abhinav', '.'],
 ['i', 'am', 'an', 'undergraduate', 'student', '.'],
 ['i', 'am', 'in', '3rd', 'year', '.']]

In [7]:
def get_tokenized_data(data):
    sentences = split_to_sentences(data)
    tokenized_sentences = tokenize_sentences(sentences)
    
    return tokenized_sentences

In [8]:
x = "I am Abhinav.\nI am an Undergraduate Student\nI am in 3rd Year."
get_tokenized_data(x)

[['i', 'am', 'abhinav', '.'],
 ['i', 'am', 'an', 'undergraduate', 'student'],
 ['i', 'am', 'in', '3rd', 'year', '.']]

# Splitting into Train-Test Sets

In [9]:
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

In [10]:
print("{} data are split into {} train and {} test set".format(len(tokenized_data), len(train_data), len(test_data)))
print("First training sample:")
print(train_data[0])      

print("First test sample")
print(test_data[0])

47961 data are split into 38368 train and 9593 test set
First training sample:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production']
First test sample
['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']


In [11]:
def count_words(tokenized_sentences):
    word_counts = {}
   
    for sentence in tokenized_sentences: 
        for token in sentence: 
            if token not in word_counts.keys(): 
                word_counts[token] = 1
            else:
                word_counts[token] += 1
    
    return word_counts

In [12]:
tokenized_sentences = [['i', 'am', 'abhinav', '.'],
 ['i', 'am', 'an', 'undergraduate', 'student', '.'],
 ['i', 'am', 'in', '3rd', 'year', '.']]

count_words(tokenized_sentences)

{'i': 3,
 'am': 3,
 'abhinav': 1,
 '.': 3,
 'an': 1,
 'undergraduate': 1,
 'student': 1,
 'in': 1,
 '3rd': 1,
 'year': 1}

# Out-of-Vocabulary Words

In [13]:
def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    closed_vocab = []
    word_counts = count_words(tokenized_sentences)
    
    for word, cnt in word_counts.items():
        if cnt >= count_threshold:
            closed_vocab.append(word)
    
    return closed_vocab

In [14]:
tokenized_sentences = [['i', 'am', 'abhinav', '.'], ['i', 'am', 'an', 'undergraduate', 'student', '.'], ['i', 'am', 'in', '3rd', 'year', '.']]
tmp_closed_vocab = get_words_with_nplus_frequency(tokenized_sentences, count_threshold = 2)

print(f"Closed vocabulary:")
print(tmp_closed_vocab)

Closed vocabulary:
['i', 'am', '.']


In [15]:
def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):
    vocabulary = set(vocabulary)
    replaced_tokenized_sentences = []
    
    for sentence in tokenized_sentences:
        replaced_sentence = []

        for token in sentence: 
            if token in vocabulary: 
                replaced_sentence.append(token)
            else:
                replaced_sentence.append(unknown_token)
  
        replaced_tokenized_sentences.append(replaced_sentence)
        
    return replaced_tokenized_sentences

In [16]:
tokenized_sentences = [["fast", "runnning"], ["slow", "walking"]]
vocabulary = ["Fast", "walking"]
tmp_replaced_tokenized_sentences = replace_oov_words_by_unk(tokenized_sentences, vocabulary)

print(f"Original sentence:")
print(tokenized_sentences)

print(f"tokenized_sentences with less frequent words converted to '<unk>':")
print(tmp_replaced_tokenized_sentences)

Original sentence:
[['fast', 'runnning'], ['slow', 'walking']]
tokenized_sentences with less frequent words converted to '<unk>':
[['<unk>', '<unk>'], ['<unk>', 'walking']]


# Pre-processing the Train-Test Data

In [17]:
def preprocess_data(train_data, test_data, count_threshold):
    vocabulary = get_words_with_nplus_frequency(train_data, count_threshold)
    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)
    
    return train_data_replaced, test_data_replaced, vocabulary

In [25]:
tmp_train = [['i', 'am', 'Abhinav', '.'], ['i', 'am', 'an','undergraduate','student','.']]
tmp_test = [['i', 'am', 'in', '3rd','Year','.']]
tmp_train_repl, tmp_test_repl, tmp_vocab = preprocess_data(tmp_train, tmp_test, count_threshold = 1)

print("tmp_train_repl")
print(tmp_train_repl)
print()

print("tmp_test_repl")
print(tmp_test_repl)
print()

print("tmp_vocab")
print(tmp_vocab)

tmp_train_repl
[['i', 'am', 'Abhinav', '.'], ['i', 'am', 'an', 'undergraduate', 'student', '.']]

tmp_test_repl
[['i', 'am', '<unk>', '<unk>', '<unk>', '.']]

tmp_vocab
['i', 'am', 'Abhinav', '.', 'an', 'undergraduate', 'student']


In [26]:
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, test_data, minimum_freq)

In [27]:
print("First preprocessed training sample:")
print(train_data_processed[0])
print()
print("First preprocessed test sample:")
print(test_data_processed[0])
print()
print("First 10 vocabulary:")
print(vocabulary[0:10])
print()
print("Size of vocabulary:", len(vocabulary))

First preprocessed training sample:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production']

First preprocessed test sample:
['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']

First 10 vocabulary:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the']

Size of vocabulary: 14823


# Developing an n-gram based language models

In [38]:
def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):
    n_grams = {}
    
    for sentence in data: 
        sentence = [start_token] * n + sentence + [end_token]
        sentence = tuple(sentence)
        
        for i in range(len(sentence) if n == 1 else len(sentence) - 1):
            n_gram = sentence[i : i + n]

            if n_gram in n_grams.keys(): 
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1
    
    return n_grams

In [39]:
sentences = [['i', 'like', 'a', 'chocolate'], ['this', 'candy', 'is', 'like', 'a', 'chocolate']]
print("Uni-gram:")
print(count_n_grams(sentences, 1))

print("Bi-gram:")
print(count_n_grams(sentences, 2))

Uni-gram:
{('<s>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('chocolate',): 2, ('<e>',): 2, ('this',): 1, ('candy',): 1, ('is',): 1}
Bi-gram:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'like'): 1, ('like', 'a'): 2, ('a', 'chocolate'): 2, ('chocolate', '<e>'): 2, ('<s>', 'this'): 1, ('this', 'candy'): 1, ('candy', 'is'): 1, ('is', 'like'): 1}


In [40]:
def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k = 1.0):
    previous_n_gram = tuple(previous_n_gram)
    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
    
    denominator = previous_n_gram_count + k * vocabulary_size
    n_plus1_gram = previous_n_gram + (word,)
    
    n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts else 0
    
    numerator = n_plus1_gram_count + k
    probability = numerator / denominator
 
    return probability

In [57]:
sentences = [['i', 'like', 'a', 'chocolate'], ['this', 'candy', 'is', 'like', 'a', 'chocolate']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
tmp_prob = estimate_probability("chocolate", "a", unigram_counts, bigram_counts, len(unique_words), k = 1)

print(f"The estimated probability of word 'chocolate' given the previous n-gram 'a' is: {tmp_prob:.4f}")

The estimated probability of word 'chocolate' given the previous n-gram 'a' is: 0.3333


## Estimating probabilities for all words

In [45]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k = 1.0):
    previous_n_gram = tuple(previous_n_gram)
    vocabulary = vocabulary + ["<e>", "<unk>"]
    vocabulary_size = len(vocabulary)
    probabilities = {}
    
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k = k)
        probabilities[word] = probability

    return probabilities

In [46]:
sentences = [['i', 'like', 'a', 'chocolate'], ['this', 'candy', 'is', 'like', 'a', 'chocolate']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_probabilities("a", unigram_counts, bigram_counts, unique_words, k = 1)

{'i': 0.09090909090909091,
 'candy': 0.09090909090909091,
 'this': 0.09090909090909091,
 'a': 0.09090909090909091,
 'chocolate': 0.2727272727272727,
 'like': 0.09090909090909091,
 'is': 0.09090909090909091,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

In [47]:
trigram_counts = count_n_grams(sentences, 3)
estimate_probabilities(["<s>", "<s>"], bigram_counts, trigram_counts, unique_words, k = 1)

{'i': 0.18181818181818182,
 'candy': 0.09090909090909091,
 'this': 0.18181818181818182,
 'a': 0.09090909090909091,
 'chocolate': 0.09090909090909091,
 'like': 0.09090909090909091,
 'is': 0.09090909090909091,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

## Count and probability matrices

In [48]:
def make_count_matrix(n_plus1_gram_counts, vocabulary):
    vocabulary = vocabulary + ["<e>", "<unk>"]
    n_grams = []
    
    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
        
    n_grams = list(set(n_grams))
    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    col_index = {word:j for j, word in enumerate(vocabulary)}
    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    
    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count
    
    count_matrix = pd.DataFrame(count_matrix, index = n_grams, columns = vocabulary)
    
    return count_matrix

In [51]:
sentences = [['i', 'like', 'a', 'chocolate'], ['this', 'candy', 'is', 'like', 'a', 'chocolate']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)
print('bi-gram counts: ')
display(make_count_matrix(bigram_counts, unique_words))

bi-gram counts


,i,candy,this,a,chocolate,like,is,<e>,<unk>
"(chocolate,)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(this,)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(candy,)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"(is,)",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(i,)",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(like,)",0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
"(<s>,)",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(a,)",0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


In [85]:
print('tri-gram counts: ')
trigram_counts = count_n_grams(sentences, 3)
display(make_count_matrix(trigram_counts, unique_words))

tri-gram counts


,i,candy,this,a,chocolate,like,is,<e>,<unk>
"(<s>, <s>)",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(like, a)",0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
"(<s>, i)",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(chocolate,)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(i, like)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(a, chocolate)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(<s>, this)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(candy, is)",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(this, candy)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"(is, like)",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [55]:
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis = 1), axis = 0)
    
    return prob_matrix

In [56]:
sentences = [['i', 'like', 'a', 'chocolate'], ['this', 'candy', 'is', 'like', 'a', 'chocolate']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)
print("bigram probabilities")
display(make_probability_matrix(bigram_counts, unique_words, k = 1))

bigram probabilities


,i,candy,this,a,chocolate,like,is,<e>,<unk>
"(chocolate,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(this,)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(candy,)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000
"(is,)",0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000
"(i,)",0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000
"(like,)",0.090909,0.090909,0.090909,0.272727,0.090909,0.090909,0.090909,0.090909,0.090909
"(<s>,)",0.181818,0.090909,0.181818,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(a,)",0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909,0.090909,0.090909


In [58]:
print("trigram probabilities")
trigram_counts = count_n_grams(sentences, 3)
display(make_probability_matrix(trigram_counts, unique_words, k = 1))

trigram probabilities


,i,candy,this,a,chocolate,like,is,<e>,<unk>
"(<s>, <s>)",0.181818,0.090909,0.181818,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(like, a)",0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909,0.090909,0.090909
"(<s>, i)",0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000
"(chocolate,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(i, like)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000
"(a, chocolate)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(<s>, this)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(candy, is)",0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000
"(this, candy)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000
"(is, like)",0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000


# Finding the Perplexity Score

In [59]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k = 1.0):
    n = len(list(n_gram_counts.keys())[0])
    sentence = ["<s>"] * n + sentence + ["<e>"]
    sentence = tuple(sentence)
    N = len(sentence)
    product_pi = 1.0
    
    for t in range(n, N):
        n_gram = sentence[t - n : t]
        word = sentence[t]
        probability = estimate_probability(word, n_gram, n_gram_counts, n_plus1_gram_counts, len(unique_words), k = 1)
        product_pi *= 1 / probability

    perplexity = product_pi ** (1/float(N))

    return perplexity

In [60]:
sentences = [['i', 'like', 'a', 'chocolate'], ['this', 'candy', 'is', 'like', 'a', 'chocolate']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

perplexity_train1 = calculate_perplexity(sentences[0], unigram_counts, bigram_counts, len(unique_words), k = 1.0)
print(f"Perplexity for first train sample: {perplexity_train1:.2f}")

test_sentence = ['i', 'like', 'a', 'candy']
perplexity_test = calculate_perplexity(test_sentence, unigram_counts, bigram_counts, len(unique_words), k = 1.0)
print(f"Perplexity for test sample: {perplexity_test:.2f}")

Perplexity for first train sample: 2.8040
Perplexity for test sample: 3.9654


# Building the system

In [61]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k = 1.0, start_with = None):
    n = len(list(n_gram_counts.keys())[0]) 
    previous_n_gram = previous_tokens[-n:]
    probabilities = estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k = k)
    suggestion = None
    max_prob = 0
    
    for word, prob in probabilities.items(): 
        if start_with != None:
            if not word.startswith(start_with): 
                continue 
        if prob > max_prob:
            suggestion = word
            max_prob = prob
    
    return suggestion, max_prob

In [62]:
sentences = [['i', 'like', 'a', 'chocolate'], ['this', 'candy', 'is', 'like', 'a', 'chocolate']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k = 1.0)

print(f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.2f}")
print()

tmp_starts_with = 'c'
tmp_suggest2 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k = 1.0, start_with = tmp_starts_with)
print(f"The previous words are 'i like', the suggestion must start with `{tmp_starts_with}`\n\tand the suggested word is `{tmp_suggest2[0]}` with a probability of {tmp_suggest2[1]:.2f}")

The previous words are 'i like',
	and the suggested word is `a` with a probability of 0.2727

The previous words are 'i like', the suggestion must start with `c`
	and the suggested word is `candy` with a probability of 0.0909


In [63]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k = 1.0, start_with = None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]
        suggestion = suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k = k, start_with = start_with)
        suggestions.append(suggestion)
        
    return suggestions

In [64]:
sentences = [['i', 'like', 'a', 'chocolate'], ['this', 'candy', 'is', 'like', 'a', 'chocolate']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
quadgram_counts = count_n_grams(sentences, 4)
qintgram_counts = count_n_grams(sentences, 5)

n_gram_counts_list = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]
previous_tokens = ["i", "like"]
tmp_suggest3 = get_suggestions(previous_tokens, n_gram_counts_list, unique_words, k = 1.0)

print(f"The previous words are 'i like', the suggestions are:")
display(tmp_suggest3)

The previous words are 'i like', the suggestions are:


[('a', 0.2727272727272727),
 ('a', 0.2),
 ('i', 0.1111111111111111),
 ('i', 0.1111111111111111)]

## Suggesting multiple words with variable lengths 

In [65]:
n_gram_counts_list = []

for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...


In [71]:
previous_tokens = ["where", "is", "the"]
tmp_suggest4 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k = 1.0)
print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest4)

The previous words are ['where', 'is', 'the'], the suggestions are:


[('<unk>', 0.037023496959021567),
 ('best', 0.003082371458551941),
 ('best', 0.00020227901018137685),
 ('i', 6.745362563237774e-05)]

In [84]:
previous_tokens = ["what", "is", "a"]
tmp_suggest5 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k = 1.0, start_with = 'm')
print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest5)

The previous words are ['what', 'is', 'a'], the suggestions are:


[('man', 0.002103474778528377),
 ('must', 0.0003935716628402755),
 ('manufacturers', 6.743088334457182e-05),
 ('manufacturers', 6.745362563237774e-05)]

In [69]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest6 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k = 1.0)
print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest6)

The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[("'re", 0.0239720461563465),
 ('?', 0.002888086642599278),
 ('?', 0.001613228473482557),
 ('<e>', 0.00013489815189531904)]

In [70]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest8 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k = 1.0, start_with = "d")
print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest8)

The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[('do', 0.00901999024865919),
 ('doing', 0.0016409583196586807),
 ('doing', 0.0004705249714324124),
 ('dvd', 6.744907594765952e-05)]